# Jared Godar Regression Project

This is the overall working notebook used to acquire / prepare / clean / scale / and explore my zillo data.

The modeling and evaluation portion will be in a second notebook `zillo-modeling.ipynb`

Streamlined highlights from both notebooks can be found in the `zillo-report.ipynb` notebook.

Import libraries used in project.

In [100]:
# Basic libraries
import pandas as pd
import numpy as np 

#Vizualization Tools
import matplotlib.pyplot as plt
import seaborn as sns

#Modeling Tools
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import statsmodels.api as sm

from datetime import date
from scipy import stats


## Evaluation tools
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from math import sqrt

import warnings
warnings.filterwarnings("ignore")

#Custim functions
from env import host, user, password #Database credentials
import zillo_wrangle
import z_wrangle2
import z_wrangle3
import eval_model




---

## Acquire

In [101]:
# function to contact database
def get_db_url(db_name):
    return f"mysql+pymysql://{user}:{password}@{host}/{db_name}"

- Look at zillow data dictionary. 
- Import minimum features (beds, bath, tax, year, fips)
- See what other columns may prove useful in model


In [ ]:
dd = pd.read_excel('zillow_data_dictionary.xlsx')
dd

### Takeaways: Columns of interest that may have predictive value

- `buildingqualitytypeid` Quality 
- `fireplacecnt`
- `garagecarcnt`
- `poolcnt`
- `rawcensustractandblock`
- `censustractandblock`
- `regionidzip`
- `regionidneighborhood`
- `storytypeid`


import:
- `regionidcounty`
- 
- `

Use SQL query to get single unit (`propertylandusetypeid=261`) from May-Aug, 2017 filtering for non-zero values to have fewer nulls in the first data pull to deal with.

In [ ]:
def get_data_from_sql():
    query = """
    SELECT bedroomcnt as bedrooms, 
       bathroomcnt as bathrooms,
       calculatedfinishedsquarefeet as square_feet,
       yearbuilt as year,
       taxamount as taxes,
       taxvaluedollarcnt as home_value,
       fips as fips,
       regionidzip as zip_code
    FROM predictions_2017
     JOIN properties_2017 USING(parcelid)
    WHERE (transactiondate >= '2017-01-01' AND transactiondate <= '2017-12-31') 
        AND propertylandusetypeid = '261'
        AND bedroomcnt > 0
        AND bathroomcnt > 0
        AND calculatedfinishedsquarefeet > 0 
        AND taxamount > 0
        AND taxvaluedollarcnt > 0
        AND fips > 0
    ORDER BY fips;
    """
    df = pd.read_sql(query, get_db_url('zillow'))
    return df

In [ ]:
zillow = get_data_from_sql()
zillow.head()

In [ ]:
shape1 = zillow.shape
shape1

Count nulls by column

In [ ]:
zillow.info(null_counts=True)


In [ ]:
# get total of null values for each row
null1 = zillow.isnull().sum()
null1

Lots of missing neighborhood data... Drop that column before filtering NAs. (Actually removed this field from the SQL query - now not imported and not dropped)

- [ ] Drop city as well
- [ ] Figure out how to get that information from `fips`  

GO back to mysql workbench and see how many properties have the `single residential inferred` code 279

- 55614 for single family

- 0 records for inferred single family, so no need to include it in query

---

### Vizualize distribution and outliers

- Eliminating outliers may also reduce the null value counts

In [ ]:
plt.figure(figsize=(16, 3))

# List of columns
cols = [col for col in zillow.columns if col not in ['fips', 'year_built', 'zip_code', 'propertylandusedesc']]

for i, col in enumerate(cols):

    # i starts at 0, but plot nos should start at 1
    plot_number = i + 1 

    # Create subplot.
    plt.subplot(1, len(cols), plot_number)

    # Title with column name.
    plt.title(col)

    # Display histogram for column.
    zillow[col].hist(bins=5)

    # Hide gridlines.
    plt.grid(False)
    
    # turn off scientific notation
    #plt.ticklabel_format(useOffset=False)
    
plt.show()

In [ ]:
plt.figure(figsize=(8,4))

plt.ticklabel_format(useOffset=False, style='plain')
sns.boxplot(data=zillow.drop(columns=['fips']))

plt.show()

Lots of outliers - especially in value

In [ ]:
zillow.describe().apply(lambda s: s.apply(lambda x: format(x, 'g')))


### Remove outliers

Make remove outliers function

In [ ]:
def remove_outliers(df, k, col_list):
    ''' remove outliers specified columns in a dataframe given a user-enterd cutoff value
    '''
    
    for col in col_list:

        q1, q3 = df[col].quantile([.25, .75])  # get quartiles
        
        iqr = q3 - q1   # calculate interquartile range
        
        upper_bound = q3 + k * iqr   # get upper bound
        lower_bound = q1 - k * iqr   # get lower bound

        # return dataframe without outliers
        
        df = df[(df[col] > lower_bound) & (df[col] < upper_bound)]
        
    return df

In [ ]:
zillow.dtypes

In [ ]:
zillow = remove_outliers(zillow, 1.5, ['bedrooms', 'bathrooms', 'square_feet', 'taxes', 'home_value'])
zillow.head()

In [ ]:
shape2 = zillow.shape
print(shape1)
print(shape2)

In [ ]:
removed1 = shape1[0]-shape2[0]

In [ ]:
print(f'Original records: {shape1[0]}')
print(f'Records Removed: {removed1}')
print(f'Records remaining: {shape2[0]}')

In [ ]:
# get total of null values for each row
null2 = zillow.isnull().sum()
print(null1)
print(null2)

Reasonable number of null values copared to total records, go ahead and drop NAs

In [ ]:
# Drop NAs
zillow = zillow.dropna()

In [ ]:
shape3=zillow.shape
shape3

In [ ]:
removed2=shape2[0]-shape3[0]

In [ ]:
print(f'Original records: {shape2[0]}')
print(f'Records Removed: {removed2}')
print(f'Records remaining: {shape3[0]}')

---

### Vizualize distributions again minus outliers

In [ ]:
plt.figure(figsize=(16, 3))

# List of columns
cols = [col for col in zillow.columns if col not in ['fips', 'zip_code', 'propertylandusedesc']]

for i, col in enumerate(cols):

    # i starts at 0, but plot nos should start at 1
    plot_number = i + 1 

    # Create subplot.
    plt.subplot(1, len(cols), plot_number)

    # Title with column name.
    plt.title(col)

    # Display histogram for column.
    zillow[col].hist(bins=5)

    # Hide gridlines.
    plt.grid(False)
    
    # turn off scientific notation
    #plt.ticklabel_format(useOffset=False)
    
plt.show()

In [ ]:
plt.figure(figsize=(8,4))

plt.ticklabel_format(useOffset=False, style='plain')
sns.boxplot(data=zillow.drop(columns=['fips', 'zip_code']))

plt.show()

In [ ]:
# List of columns
cols = [col for col in zillow.columns if col not in ['fips','zip_code', 'propertylandusedesc']]
plt.figure(figsize=(16, 20))
for i, col in enumerate(cols):

    # i starts at 0, but plot nos should start at 1
    plot_number = i + 1 

    # Create subplot.
    plt.subplot(1, len(cols), plot_number)

    # Title with column name.
    plt.title(col)

    # Display boxplot for column.
    sns.boxplot(data=zillow[col])

    # Hide gridlines.
    plt.grid(False)

plt.show()

In [ ]:
zillow.dtypes

In [ ]:
# List of columns
cols = ['bedrooms', 'bathrooms', 'square_feet', 'home_value', 'taxes']

plt.figure(figsize=(8, 3))

for i, col in enumerate(cols):

    # i starts at 0, but plot should start at 1
    plot_number = i + 1 

    # Create subplot.
    plt.subplot(1, len(cols), plot_number)

    # Title with column name.
    plt.title(col)

    # Display boxplot for column.
    sns.boxplot(data=zillow[[col]])

    # Hide gridlines.
    plt.grid(False)

    # sets proper spacing between plots
    plt.tight_layout()
    
plt.show()

In [ ]:
# function to clean up my zillow df
def clean_data(df):
    '''
    This funciton takes in the zillow df and drops observations with Null values
    and handles data types returning a df with a basic clean.
    '''
    df = df.dropna()
    df['fips'] = df['fips'].astype(int)
    df['zip_code'] = df['zip_code'].astype(int)
    df['square_feet'] = df['square_feet'].astype(int)
    df['year'] = df['year'].astype(int)

    return df

In [ ]:
zillow.shape

In [ ]:
zillow = clean_data(zillow)

In [ ]:
zillow.shape

In [ ]:
zillow = clean_data(zillow)
print(zillow.shape)
zillow.head()

In [ ]:
zillow.describe().T

### To Do in successve iterations beyond MVP

- [ ] (for unchecked checkbox)
- [x] (for checked checkbox)


- [ ] Add column for range...
- [ ] Import additional columns of potential use
- [ ] Derive columns from there
    - Pool (boolean)
    - Condition (bins)
    - Calculate age in years
    - Bin ages
    - Etc.
- [ ] Lookup / populate county based on `fips`
- [ ] Caculate tax rate percent (`taxes`, `home_value`)


-[] Add "inferred single family residential" code to original SQL query (not necessary, 0 records)

- [x] left join on propertylandusetype



Minor, but kind of annoying find out why `[ ]` is not rendering as a checkbox in markdown in VS Code...

---

## County Data for Question

Fips codes can be found [here](https://www.nrcs.usda.gov/wps/portal/nrcs/detail/national/home/?cid=nrcs143_013697)

What fips are used in dataset?

In [ ]:
zillow.fips.value_counts()

In [ ]:
29295+11806+3821

In [ ]:
zillow.shape

### Fips values 

| fips |   County    | State |
| :--: | :---------: | :---: |
| 6037 | Los Angeles |  CA   |
| 6059 |   Orange    |  CA   |
| 6111 |   Ventura   |  CA   |

In [ ]:
zillow.head()

In [ ]:
zillow.dtypes

In [ ]:
# Define function to add county based on fips by row

def assign_county(row):
    if row['fips']==6037:
        return 'Los Angeles'
    if row['fips']==6059:
        return 'Orange'
    if row['fips']==6111:
        return 'Ventura'

In [ ]:
#Use function to assign county

zillow['county'] = zillow.apply(lambda row: assign_county(row), axis =1)

In [ ]:
#Add state columns

zillow['state'] = 'CA'

In [ ]:
zillow.sample(50)

In [ ]:
## COnvert year into integer (May delete or comment out later since I added this to an earlier function)

zillow['year'] = zillow['year'].astype(int)


In [ ]:
# Confrim year dtype changed

zillow.dtypes

### Add county averages to DF

In [ ]:
# Split into 3 county dataframes

la = zillow[zillow.county=='Los Angeles']
oc = zillow[zillow.county=='Orange']
ven = zillow[zillow.county=='Ventura']

In [ ]:
# Calculate average home price by county

la_avg = la.home_value.mean()
oc_avg = oc.home_value.mean()
ven_avg = ven.home_value.mean()


In [ ]:
# Add average county price as column

def assign_county_avg(row):
    if row['fips']==6037:
        return la_avg
    if row['fips']==6059:
        return oc_avg
    if row['fips']==6111:
        return ven_avg

In [ ]:
zillow['county_avg'] = zillow.apply(lambda row: assign_county_avg(row), axis =1)

In [ ]:
zillow.sample(25)

---

### Feature engineering

- Will explore more later, but any initial features?
- Transform year built to age

In [ ]:
from datetime import date


In [ ]:
## add age column
zillow['age'] = date.today().year-zillow.year

In [ ]:
# Confirm

zillow.head()

## Location Data

- One-hot encode county so I can pass that to models

In [ ]:
# Add this to clean module

dummy_df = pd.get_dummies(zillow[['county']], drop_first=True)
zillow = pd.concat([zillow, dummy_df], axis=1)

In [ ]:
zillow.sample(25)

### Add Tax rate column

taxes = home_value * tax_rate

tax_rate = taxes / home_value

In [ ]:
zillow['tax_rate']=round(((zillow.taxes/zillow.home_value)*100),2)

In [ ]:
zillow.sample(25)

In [ ]:
zillow.tax_rate.min()

In [ ]:
zillow.tax_rate.max()

---

## Split data into train, test, validate; Then create separate x/y feature/target dataframes


In [ ]:
def split_my_data(df, pct=0.10):
    '''
    This splits a dataframe into train, validate, and test sets. 
    df = dataframe to split
    pct = size of the test set, 1/2 of size of the validate set
    Returns three dataframes (train, validate, test)
    '''
    train_validate, test = train_test_split(df, test_size=pct, random_state = 123)
    train, validate = train_test_split(train_validate, test_size=pct*2, random_state = 123)
    return train, validate, test

In [ ]:
train, validate, test = split_my_data(zillow)

In [ ]:
print(train.shape)
print(validate.shape)
print(test.shape)

## Baseline

- Can go ahead and add a baseline, y_hat prediction
- Will use the training median as baseline, less sensitive to outliers.

In [ ]:
## Baseline

baseline = train.home_value.median()
baseline


In [ ]:
train['baseline'] = baseline
validate['baseline'] = baseline
test['baseline'] = baseline

In [ ]:
train.head()

In [ ]:
# Split into x / y | features / target

# Setup X and y
X_train = train.drop(columns='home_value')
y_train = train.home_value

X_validate = validate.drop(columns='home_value')
y_validate = validate.home_value

X_test = test.drop(columns='home_value')
y_test = test.home_value

---

## Data has been aquired and cleaned, now scale

Since even our cleaned data has a fair number of outliers still, I will use the robust scaler

### Beginning exploration

- Examine pairwaise relationships
    - Crosstabs
    - Corr plots
    - Pair plots
    - Etc.

In [ ]:
import sklearn.preprocessing
from sklearn.model_selection import train_test_split


In [ ]:
train.head()

In [ ]:
train.dtypes

**NOTE:** I originally scaled `fips` and `zip_code` because I wanted to pass more granular location data to my model. However, using a scaled value imploes a numberic relationship between the values (90210 being 100 more somethings from 90110, which is not the case). So, I am eliminating them from the scaler.

- To get in more granular location data, in a sucessive iteration I will one-hot encode the counties to pass along to the model

- To dive in finer:
    - Pull lat lon from the original database and see how many nulls there are
    - Convert zip to lat lon
    - Use an unsupervised clustering algorithm to create k neighborhoods
    - Explore approppriate k for number of obeservations; 3-6?

In [ ]:
# Fit scaler to training data

scaler = sklearn.preprocessing.RobustScaler()

columns = ['bedrooms', 'bathrooms', 'square_feet',  'age']

scaler.fit(X_train[columns])


In [ ]:
# Apply scaler to all data

new_column_names = [c + '_scaled' for c in columns]

X_train = pd.concat([X_train, pd.DataFrame(scaler.transform(X_train[columns]), columns=new_column_names, index = train.index),], axis=1)

X_validate = pd.concat([X_validate, pd.DataFrame(scaler.transform(X_validate[columns]), columns=new_column_names, index = validate.index),], axis=1)

X_test = pd.concat([X_test, pd.DataFrame(scaler.transform(X_test[columns]), columns=new_column_names, index = test.index),], axis=1)



In [ ]:
X_train.sample(25)

In [ ]:
X_train.shape

In [ ]:
#Vizualize scaler

plt.figure(figsize=(13, 6))
plt.subplot(121)
plt.hist(X_train.square_feet, bins=25, ec='black')
plt.title('Original')
plt.subplot(122)
plt.hist(X_train.square_feet_scaled, bins=25, ec='black')
plt.title('Scaled')

In [ ]:
#Vizualize scaler

plt.figure(figsize=(13, 6))
plt.subplot(121)
plt.hist(X_train.bedrooms, bins=25, ec='black')
plt.title('Original')
plt.subplot(122)
plt.hist(X_train.bedrooms_scaled, bins=25, ec='black')
plt.title('Scaled')

In [ ]:
#Vizualize scaler

plt.figure(figsize=(13, 6))
plt.subplot(121)
plt.hist(X_train.bathrooms, bins=25, ec='black')
plt.title('Original')
plt.subplot(122)
plt.hist(X_train.bathrooms_scaled, bins=25, ec='black')
plt.title('Scaled')

In [ ]:
#Vizualize scaler

plt.figure(figsize=(13, 6))
plt.subplot(121)
plt.hist(X_train.age, bins=25, ec='black')
plt.title('Original')
plt.subplot(122)
plt.hist(X_train.age_scaled, bins=25, ec='black')
plt.title('Scaled')

In [ ]:
X_train.describe().T

___

In [ ]:
########## Combine all of the above steps into function(s)#################

#################### IMPORT LIBRARIES #################

# Basic libraries
import pandas as pd
import numpy as np 

#Vizualization Tools
import matplotlib.pyplot as plt
import seaborn as sns

#Modeling Tools
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import statsmodels.api as sm

from datetime import date

import sklearn.preprocessing
from sklearn.model_selection import train_test_split


import warnings
warnings.filterwarnings("ignore")

#Custim functions
from env import host, user, password #Database credentials
import zillo_wrangle





################ PULL DATA FROM DB ############## 

def get_db_url(db_name):
    return f"mysql+pymysql://{user}:{password}@{host}/{db_name}"


def get_data_from_sql():
    query = """
    SELECT bedroomcnt as bedrooms, 
       bathroomcnt as bathrooms,
       calculatedfinishedsquarefeet as square_feet,
       yearbuilt as year,
       taxamount as taxes,
       taxvaluedollarcnt as home_value,
       fips as fips,
       regionidzip as zip_code
    FROM predictions_2017
     JOIN properties_2017 USING(parcelid)
    WHERE (transactiondate >= '2017-01-01' AND transactiondate <= '2017-12-31') 
        AND propertylandusetypeid = '261'
        AND bedroomcnt > 0
        AND bathroomcnt > 0
        AND calculatedfinishedsquarefeet > 0 
        AND taxamount > 0
        AND taxvaluedollarcnt > 0
        AND fips > 0
    ORDER BY fips;
    """
    df = pd.read_sql(query, get_db_url('zillow'))
    return df


################ REMOVE OUTLIERS #################

def remove_outliers(df, k, col_list):
    ''' remove outliers from a list of columns in a dataframe 
        and return that dataframe
    '''
    
    for col in col_list:

        q1, q3 = df[col].quantile([.25, .75])  # get quartiles
        
        iqr = q3 - q1   # calculate interquartile range
        
        upper_bound = q3 + k * iqr   # get upper bound
        lower_bound = q1 - k * iqr   # get lower bound

        # return dataframe without outliers
        
        df = df[(df[col] > lower_bound) & (df[col] < upper_bound)]
        
    return df

######## Clean Data ###########

def clean_data(df):
    '''
    This funciton takes in the zillow df and drops  Null values reassigns some dtypes.
    '''
    df = df.dropna()
    df['fips'] = df['fips'].astype(int)
    df['zip_code'] = df['zip_code'].astype(int)
    df['square_feet'] = df['square_feet'].astype('int')
    df['year'] = df['year'].astype(int)

    return df

######### ADD COUNTY AND STATE COLUMNS #######

def assign_county(row):
    if row['fips']==6037:
        return 'Los Angeles'
    if row['fips']==6059:
        return 'Orange'
    if row['fips']==6111:
        return 'Ventura'

######## Feature engineering ########

def engineer(zillow):
    zillow['county'] = zillow.apply(lambda row: assign_county(row), axis =1) #Add counties
    zillow['state'] = 'CA' #Add state
    zillow['age'] = date.today().year-zillow.year # Add age
    dummy_df = pd.get_dummies(zillow[['county']], drop_first=True)
    zillow = pd.concat([zillow, dummy_df], axis=1)
    return zillow

###### ADD COUNTY AVERAGE #############

def county_avg(zillow):
    la = zillow[zillow.county=='Los Angeles']
    oc = zillow[zillow.county=='Orange']
    ven = zillow[zillow.county=='Ventura']

    la_avg = la.home_value.mean()
    oc_avg = oc.home_value.mean()
    ven_avg = ven.home_value.mean()

    def assign_county_avg(row):
        if row['fips']==6037:
            return la_avg
        if row['fips']==6059:
            return oc_avg
        if row['fips']==6111:
            return ven_avg

    zillow['county_avg'] = zillow.apply(lambda row: assign_county_avg(row), axis =1)

    return zillow

########## TRAIN VALIDATE TEST SPLIT #########

def split_my_data(df, pct=0.10):
    '''
    This splits a dataframe into train, validate, and test sets. 
    df = dataframe to split
    pct = size of the test set, 1/2 of size of the validate set
    Returns three dataframes (train, validate, test)
    '''
    train_validate, test = train_test_split(df, test_size=pct, random_state = 123)
    train, validate = train_test_split(train_validate, test_size=pct*2, random_state = 123)
    return train, validate, test

########## ADD BASELINE #########

def add_baseline(train, validate, test):
    baseline = train.home_value.median()
    train['baseline'] = baseline
    validate['baseline'] = baseline
    test['baseline'] = baseline
    return train, validate, test

######## SPLIT IN TO X /y features / target ########

def split_xy(train, validate, test):
    X_train = train.drop(columns='home_value')
    y_train = train.home_value

    X_validate = validate.drop(columns='home_value')
    y_validate = validate.home_value

    X_test = test.drop(columns='home_value')
    y_test = test.home_value

    return X_train, y_train, X_validate, y_validate, X_test, y_test

############## Robust Scale ###############

def scale(X_train, X_validate, X_test, train, validate, test):
    scaler = sklearn.preprocessing.RobustScaler()

    columns = ['bedrooms', 'bathrooms', 'square_feet', 'fips', 'age', 'zip_code']
    
    scaler.fit(X_train[columns])

    new_column_names = [c + '_scaled' for c in columns]

    X_train = pd.concat([X_train, pd.DataFrame(scaler.transform(X_train[columns]), columns=new_column_names, index = train.index),], axis=1)

    X_validate = pd.concat([X_validate, pd.DataFrame(scaler.transform(X_validate[columns]), columns=new_column_names, index = validate.index),], axis=1)

    X_test = pd.concat([X_test, pd.DataFrame(scaler.transform(X_test[columns]), columns=new_column_names, index = test.index),], axis=1)
    
    return X_train, X_validate, X_test

######### CALL ALL FUNCTIONS TOGETHER #######

def wrangle():
    zillow = get_data_from_sql()
    zillow = remove_outliers(zillow, 1.5, ['bedrooms', 'bathrooms', 'square_feet', 'taxes', 'home_value'])
    zillow = clean_data(zillow) #Drop NAs and change dtypes
    zillow = engineer(zillow)
    zillow = county_avg(zillow)
    train, validate, test = split_my_data(zillow)
    train, validate, test = add_baseline(train, validate, test)
    X_train, y_train, X_validate, y_validate, X_test, y_test = split_xy(train, validate, test)
    X_train, X_validate, X_test = scale(X_train, X_validate, X_test, train, validate, test)
    return X_train, y_train, X_validate, y_validate, X_test, y_test








---

## EDA

### Initial questions:

1. What are drivers of tax value?
2. What leads to lower tax values?
3. What factors do not impact tax value?
4. Is there a difference in average price by county
5. Are there any ways to combine the current data into interesting engineered features?

In [ ]:
X_train.sample(25)

In [ ]:
df= X_train

---

In [ ]:
# Quick look at distribution of all numeric columns in df

df.hist(grid=False, figsize=(16,12));




In [ ]:
df=train

In [ ]:
columns = ['bedrooms', 'bathrooms', 'square_feet', 'age', 'home_value']
sns.heatmap(df[columns].corr(), cmap='Blues', annot=True)



### Initial impressions

- Looking at factors that impact home value, Square footage seems to be the highest driver, followed by bathrooms, then bedrooms

- There is a negative correlation with age

- Bedrooms matter least out of these sparse columns

In [ ]:
from scipy import stats

In [ ]:
def correlation_exploration(df, x_string, y_string):
    r, p = stats.pearsonr(df[x_string], df[y_string])
    ax= sns.regplot(x=x_string, y=y_string, data=df, line_kws={"color": "red"})
    plt.title(f"{x_string}'s Relationship with {y_string}")
    print(f'The p-value is: {p}. There is {round(p,3)}% chance that we see these results by chance.')
    print(f'r = {round(r, 2)}')

In [ ]:
correlation_exploration(df, 'square_feet', 'home_value')

In [ ]:
correlation_exploration(df, 'bathrooms', 'home_value')

In [ ]:
correlation_exploration(df, 'bedrooms', 'home_value')

In [ ]:
correlation_exploration(df, 'age', 'home_value')

- [x]Add lines of best fit to scatterplots

Statistical tests confirm, strongest correlation with area, followed by bathrooms, then bedrooms with age being negatively corrolated.

In [ ]:
sns.histplot(data=df, x='square_feet')

In [ ]:
sns.histplot(data=df, y='bedrooms')

In [ ]:
sns.histplot(data=df, y='bathrooms')

In [ ]:
sns.histplot(data=df, x='age')

In [ ]:
df.dtypes

In [ ]:
sns.histplot(data=df, x='county')

In [ ]:

df['zip_code']=df['zip_code'].astype(int)

In [ ]:
sns.histplot(data=df, x='zip_code')

weird scaling issues - play arond more if you really care to

Is there a difference in home prices by county?

In [ ]:
# Make a separate dataframe for each county

la = train[train.county=='Los Angeles'].home_value
oc = train[train.county=='Orange'].home_value
ven = train[train.county=='Ventura'].home_value

In [ ]:
# Vizualize prices by county

ax = sns.boxplot(x=train.county, y=train.home_value)
#ax = sns.swarmplot(x=train.county, y=train.home_value, color ='.25')
plt.title('Figure 5: Average value by County')

- Perform Kruskal-Wallis H-test to see if there is a difference by county.

- Using a nonparametric test since the underlying distributions are not normal.

In [ ]:
stats.kruskal(la, oc, ven)

- There are differences by county. 

- Post-hoc, pairwaise analysis can determine which samples are different in which direction.

In [ ]:
# Is LA lower than OC
stats.mannwhitneyu(la, oc, alternative ='less')

- Los Angeles County prices are lower, on average, than Orange County 

In [ ]:
# Is LA lower than Ventura

stats.mannwhitneyu(la, ven, alternative ='less')

- Los Angeles County prices are lower, on average, than Ventura County 

In [ ]:
# Is Ventura Lower than Orange

stats.mannwhitneyu(ven, oc, alternative ='less')

Ventura is not statistically lower than orange

In [ ]:
# Is there a difference between Ventura and orange

stats.mannwhitneyu(ven, oc)

In [ ]:
stats.kruskal(oc, ven)

There is no statistically significant difference between the mean home values in Orange and Ventura Counties.

### Ideas for final figures:

- Figure 1 - Distributions: Panels with distributions of all target varibles
- Figure 2 - Drivers: panels with correlations and R/p labelled
- Figure 3 - Redce (age)
- Figure 4 - Don't matter

In [ ]:
sns.histplot(data=df, x='home_value', hue='bedrooms', palette='husl', alpha = 0.25)

In [ ]:
sns.histplot(data=df, x='home_value', hue='bathrooms', palette='husl', alpha = 0.25)

---

# Modeling

Data is now clean split, and scaled and we have a baseline and a feel for meaningful drivers. Proceed with creating MVP model.

In [ ]:
# - Import ols


from statsmodels.formula.api import ols



In [ ]:
# Create Model

ols_model = ols(formula='home_value ~ bedrooms + bathrooms + square_feet', data=train).fit()



In [ ]:
# Make predictions

ols_yhat = ols_model.predict(X_train)



In [ ]:
X_train['mvp_prdictions']=ols_yhat

In [ ]:
X_train.sample(20)

In [ ]:
y_train

---

### Evaluate model

- RMSE and R^2 will be primary evaluation metrics
- Make a dataframe with actual values, baseline predictions, and model predictions
- Calculate RMSE
- Calculate R^2

In [ ]:
#  DataFrame for evaluating model 

ols_eval = y_train.copy()
validate_eval = y_validate.copy()

In [ ]:
ols_eval = pd.DataFrame(ols_eval)
validate_eval = pd.DataFrame(validate_eval)

In [ ]:
ols_eval.rename(columns={'home_value': 'actual'}, inplace=True)
validate_eval.rename(columns={'home_value': 'actual'}, inplace=True)



In [ ]:
validate_eval

In [ ]:
# Add baseline - median home value

ols_eval['baseline_yhat'] = ols_eval['actual'].median()

validate_eval['baseline_yhat'] = ols_eval['actual'].median()



In [ ]:
# Add model prediction

ols_eval['ols_yhat'] = ols_model.predict(X_train)
validate_eval['ols_yhat'] = ols_model.predict(X_validate)



In [ ]:
validate_eval

In [ ]:
# Calculate and Add Residuals Column for Plotting

ols_eval['residuals'] = ols_eval.ols_yhat - ols_eval.actual
validate_eval['residuals'] = validate_eval.ols_yhat - validate_eval.actual



In [ ]:
ols_eval

In [ ]:
#  Compute the RMSE  and R2 for  ols model and baseline 

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from math import sqrt

baseline_RMSE = round(sqrt(mean_squared_error(ols_eval.actual, ols_eval.baseline_yhat)))
ols_RMSE = round(sqrt(mean_squared_error(ols_eval.actual, ols_eval.ols_yhat)))
pct_change=round(((ols_RMSE-baseline_RMSE)/baseline_RMSE)*100, 2)
rmse_validate = round(sqrt(mean_squared_error(validate_eval.actual, validate_eval.ols_yhat)))
baseline_r2 = round(r2_score(ols_eval.actual, ols_eval.baseline_yhat), 2)
ols_train_r2 = round(r2_score(ols_eval.actual, ols_eval.ols_yhat), 2)
ols_validate_r2 = round(r2_score(validate_eval.actual, validate_eval.ols_yhat), 2)

print(f'My model has value: {ols_RMSE < baseline_RMSE}')
print()
print(f'Baseline RMSE: {baseline_RMSE}')
print(f'My model train RMSE: {ols_RMSE}')
print(f'My model validate RMSE: {rmse_validate}')
print(f'RMSE difference baseline to model: {baseline_RMSE- ols_RMSE}')
print(f'RMSE difference train to validate: {ols_RMSE- rmse_validate}')
print(f'RMSE improvement: {pct_change}%')
print()
print(f'Baseline R2: {baseline_r2}')
print(f'Model train  R2: {ols_train_r2}')
print(f'Model Validate R2: {ols_validate_r2}')



In [ ]:
ols_model.summary()



In [ ]:
# Look at risiduals distribution

plt.hist(np.log(ols_eval.residuals));



Definitley a skew to the residuals - not centered around 0

In [ ]:
# Look at Predictions vs Residuals

plt.scatter(ols_eval.ols_yhat, ols_eval.residuals)

---

Next model: same as before, but add age

---

## OLS2: add age

In [ ]:
train.head()

In [ ]:
ols2_model = ols(formula='home_value ~ bedrooms + bathrooms + square_feet + age',  data=train).fit() #Create model
ols2_yhat = ols2_model.predict(X_train) # Make predictions
X_train['model2']=ols2_yhat


In [ ]:
# Add to eval
ols_eval['ols2_yhat'] = ols2_model.predict(X_train)
validate_eval['ols2_yhat'] = ols2_model.predict(X_validate)
ols_eval['ols2_residuals'] = ols_eval.ols2_yhat - ols_eval.actual
validate_eval['ols2_residuals'] = validate_eval.ols2_yhat - validate_eval.actual


In [ ]:
baseline_RMSE = round(sqrt(mean_squared_error(ols_eval.actual, ols_eval.baseline_yhat)))
ols_RMSE = round(sqrt(mean_squared_error(ols_eval.actual, ols_eval.ols_yhat)))
ols2_RMSE = round(sqrt(mean_squared_error(ols_eval.actual, ols_eval.ols2_yhat)))
pct_change_baseline=round(((ols_RMSE-baseline_RMSE)/baseline_RMSE)*100, 2)
pct_change_last_model=round(((ols2_RMSE-ols_RMSE)/ols_RMSE)*100, 2)
rmse_validate = round(sqrt(mean_squared_error(validate_eval.actual, validate_eval.ols2_yhat)))
baseline_r2 = round(r2_score(ols_eval.actual, ols_eval.baseline_yhat), 2)
ols2_train_r2 = round(r2_score(ols_eval.actual, ols_eval.ols2_yhat), 2)
ols2_validate_r2 = round(r2_score(validate_eval.actual, validate_eval.ols2_yhat), 2)


print(f'My model has value: {ols_RMSE < baseline_RMSE}')
print(f'My model beats previous model: {ols2_RMSE < ols_RMSE}')
print()
print(f'Baseline RMSE: {baseline_RMSE}')
print(f'Model 1 RMSE: {ols_RMSE}')
print(f'Currennt model train RMSE: {ols2_RMSE}')
print(f'Currennt model validate RMSE: {rmse_validate}')
print()
print(f'Current model RMSE difference from baseline: {baseline_RMSE- ols_RMSE}')
print(f'RMSE difference train to validate: {ols2_RMSE- rmse_validate}')
print(f'Current model baseline RMSE improvement: {pct_change_baseline}%')
print(f'Current model RMSE improvement from last model: {pct_change_last_model}%')
print()
print(f'Baseline R2: {baseline_r2}')
print(f'Model train  R2: {ols2_train_r2}')
print(f'Model Validate R2: {ols2_validate_r2}')




In [ ]:
ols_eval

In [ ]:
# Look at risiduals distribution

plt.hist(np.log(ols_eval.ols2_residuals));



In [ ]:
# Look at Predictions vs Residuals

plt.scatter(ols_eval.ols_yhat, ols_eval.ols2_residuals)

In [ ]:
ols2_model.summary()


### Model 3: add county

In [ ]:
train.head()

In [ ]:
ols3_model = ols(formula='home_value ~ bedrooms + bathrooms + square_feet + age + county_Orange + county_Ventura',  data=train).fit() #Create model
ols3_yhat = ols3_model.predict(X_train) # Make predictions
X_train['model3']=ols3_yhat

In [ ]:
ols_eval

In [ ]:
ols_eval['ols3_yhat'] = ols3_model.predict(X_train)
ols_eval['ols3_residuals'] = ols_eval.ols3_yhat - ols_eval.actual
validate_eval['ols3_yhat'] = ols3_model.predict(X_validate)
validate_eval['ols3_residuals'] = validate_eval.ols3_yhat - validate_eval.actual

In [ ]:
validate_eval

In [ ]:
ols_eval

In [ ]:
baseline_RMSE = round(sqrt(mean_squared_error(ols_eval.actual, ols_eval.baseline_yhat)))
ols_RMSE = round(sqrt(mean_squared_error(ols_eval.actual, ols_eval.ols_yhat)))
ols2_RMSE = round(sqrt(mean_squared_error(ols_eval.actual, ols_eval.ols2_yhat)))
ols3_RMSE = round(sqrt(mean_squared_error(ols_eval.actual, ols_eval.ols3_yhat)))

pct_change_baseline=round(((ols3_RMSE-baseline_RMSE)/baseline_RMSE)*100, 2)
pct_change_last_model=round(((ols3_RMSE-ols2_RMSE)/ols2_RMSE)*100, 2)

ols3_train_r2 = round(r2_score(ols_eval.actual, ols_eval.ols3_yhat), 2)
ols3_validate_r2 = round(r2_score(validate_eval.actual, validate_eval.ols3_yhat), 2)

print(f'My model has value: {ols3_RMSE < baseline_RMSE}')
print(f'My model beats previous model: {ols3_RMSE < ols2_RMSE}')
print()
print(f'Baseline RMSE: {baseline_RMSE}')
print(f'Model 1 RMSE: {ols_RMSE}')
print(f'Model 2 RMSE: {ols2_RMSE}')
print(f'Current model RMSE: {ols3_RMSE}')
print()
print(f'Current model RMSE difference from baseline: {baseline_RMSE- ols3_RMSE}')
print(f'Current model baseline RMSE improvement: {pct_change_baseline}%')
print(f'Current model RMSE improvement from last model: {pct_change_last_model}%')
print()
print(f'Baseline R2: {baseline_r2}')
print(f'Current Model train  R2: {ols3_train_r2}')
print(f'Current Model Validate R2: {ols3_validate_r2}')


In [ ]:
# Look at risiduals distribution

plt.hist(np.log(ols_eval.ols3_residuals));



In [ ]:
# Look at Predictions vs Residuals

plt.scatter(ols_eval.ols_yhat, ols_eval.ols3_residuals)

In [ ]:
ols3_model.summary()


RMSD went down firther and R^2 increased

[] Add RMSE for validate on model 3

---

### Takeaways / Feature Engineering

- MVP model beats baseline
- Now, try to beat model
- How? Feature engineering
- Potential features:
    - Incorporate location:
        - One-hot encoded counties for now
        - Future iterations: Cluster lat/lon for more granular location detail
    - Add boolean columns:
        - Garage / no garage
        - Pool / no pool
        - Etc.
    - Room score: some kind of single value incorporating both the bedrooms and bathrooms
        - Maybe multiplied / weighted by their correlation coefficients
    - Stratify columns:
        - Age: historical, new constuction, in between

- Build new, select * query to bring in more fields to use for engineering

### EVALUATION VIZUALS

- [] Better actual vs predicted plots
    - For a small enough sample of values you can see them
- []Residual plots
- [] Histograms of actual vs predicted

---

### LASSO / LARS

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
# create the model object
lars = LassoLars(alpha=1.0)

In [ ]:
y_train=pd.DataFrame(y_train)

In [ ]:
columns =['bedrooms_scaled', 'bathrooms_scaled','square_feet_scaled', 'age_scaled', 'county_Orange', 'county_Ventura']

In [ ]:
X_train.dtypes

In [ ]:
y_train

In [ ]:

lasso_lars = lars.fit(X_train[columns], y_train.home_value)

In [ ]:
y_train['lars_predict']=lars.predict(X_train[columns])

In [ ]:
rmse_train = sqrt(mean_squared_error(y_train.home_value, y_train.lars_predict))


In [ ]:
rmse_train

Basely worse than model 3 above by RMSE, check R^2

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
lars_r2 =r2_score(y_train.home_value, y_train.lars_predict)

In [ ]:
lars_r2

# TO DO 

- [X] Add R2 to printed summaries
- [] Hard Code Datafram of models with rmse and r2 for easy, side-by-side comparison

---

Using a separate wrangle file to feature engineer a more robust model beyond my MVP without breaking anything in that,

### Plan:
1. Use the same query - change only scalers
    -  Linear & Standard
    - See if it improves model
2. Import all the fields from the database, clean them up, and use K-best for modeling
    - Rename columns
    - Drop Columns
    - Engineer columns
    - Encode columns
    - Model

## Min-Max scaler

In [ ]:
import z_wrangle2

In [ ]:
train_mm, X_train_mm, y_train_mm, X_validate_mm, y_validate_mm, X_test_mm, y_test_mm = z_wrangle2.wrangle()

Test this on current best performing model and compare to current performance

In [ ]:
# Make model and predictions
ols3_model = ols(formula='home_value ~ bedrooms + bathrooms + square_feet + age + county_Orange + county_Ventura',  data=train).fit() #Create model
ols3_yhat_mm = ols3_model.predict(X_train_mm) # Make predictions
X_train['model3_mm']=ols3_yhat
ols_eval['ols3_yhat_mm'] = ols3_model.predict(X_train_mm)
ols_eval['ols3_residuals_mm'] = ols_eval.ols3_yhat_mm - ols_eval.actual
validate_eval['ols3_yhat_mm'] = ols3_model.predict(X_validate_mm)
validate_eval['ols3_residuals_mm'] = validate_eval.ols3_yhat_mm - validate_eval.actual
# Calculate evaluation metrics
baseline_RMSE = round(sqrt(mean_squared_error(ols_eval.actual, ols_eval.baseline_yhat)))
ols_RMSE = round(sqrt(mean_squared_error(ols_eval.actual, ols_eval.ols_yhat)))
ols2_RMSE = round(sqrt(mean_squared_error(ols_eval.actual, ols_eval.ols2_yhat)))
ols3_RMSE = round(sqrt(mean_squared_error(ols_eval.actual, ols_eval.ols3_yhat)))
ols3_RMSE_mm = round(sqrt(mean_squared_error(ols_eval.actual, ols_eval.ols3_yhat_mm)))


pct_change_baseline=round(((ols3_RMSE_mm-baseline_RMSE)/baseline_RMSE)*100, 2)
pct_change_last_model=round(((ols3_RMSE_mm-ols3_RMSE)/ols3_RMSE)*100, 2)

ols3_train_r2_mm = round(r2_score(ols_eval.actual, ols_eval.ols3_yhat_mm), 2)
ols3_validate_r2_mm = round(r2_score(validate_eval.actual, validate_eval.ols3_yhat_mm), 2)
# Display findings
print(f'My model has value: {ols3_RMSE_mm < baseline_RMSE}')
print(f'My model beats previous model: {ols3_RMSE_mm < ols3_RMSE}')
print()
print(f'Baseline RMSE: {baseline_RMSE}')
print(f'Model 1 RMSE: {ols_RMSE}')
print(f'Model 2 RMSE: {ols2_RMSE}')
print(f'Model 3 RMSE: {ols3_RMSE}')
print(f'Current model RMSE: {ols3_RMSE_mm}')
print()
print(f'Current model RMSE difference from baseline: {baseline_RMSE- ols3_RMSE_mm}')
print(f'Current model baseline RMSE improvement: {pct_change_baseline}%')
print(f'Current model RMSE improvement from last model: {pct_change_last_model}%')
print()
print(f'Baseline R2: {baseline_r2}')
print(f'Current Model train  R2: {ols3_train_r2}')
print(f'Current Model Validate R2: {ols3_validate_r2}')


The robust scaler and min max scaler produced the same results. 

[ ] If time, Try again with standard scaler. For now, go to full database pull snd feature engineering.

---

Before doing the heavy-lifting feature engineering

- [] first try a polynomial model 
- [] Tweedie
    - power 0, 1, (1,2), 2, 3
- [] Re-run the Lasso with different $\alpha$ values

## Polynomial models

In [171]:
train, X_train, y_train, X_validate, y_validate, X_test, y_test = zillo_wrangle.wrangle()

In [174]:
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor


In [177]:
from sklearn.preprocessing import PolynomialFeatures


In [178]:
columns=['bedrooms', 'bathrooms', 'square_feet', 'age']

# make the polynomial features to get a new set of features
pf = PolynomialFeatures(degree=2)

# fit and transform X_train_scaled
X_train_degree2 = pf.fit_transform(X_train[columns])

# transform X_validate_scaled & X_test_scaled
X_validate_degree2 = pf.transform(X_validate[columns])


In [179]:
# create the model object
lm2 = LinearRegression(normalize=True)


In [183]:
xtr_df=pd.DataFrame(X_train_degree2)

In [184]:


# fit the model to our training data. We must specify the column in y_train, 
# since we have converted it to a dataframe from a series! 
lm2.fit(X_train_degree2, y_train)


LinearRegression(normalize=True)

In [187]:
y_train=pd.DataFrame(y_train)

In [188]:

# predict train
y_train['pred_lm2'] = lm2.predict(X_train_degree2)


In [193]:

# evaluate: rmse
rmse_train = mean_squared_error(y_train.home_value, y_train.pred_lm2)**(1/2)


In [194]:
y_validate=pd.DataFrame(y_validate)

In [197]:

# predict validate
y_validate['pred_lm2'] = lm2.predict(X_validate_degree2)

# evaluate: rmse
rmse_validate = mean_squared_error(y_validate.home_value, y_validate.pred_lm2)**(1/2)

lm2_validate_r2 = round(r2_score(y_validate.home_value, y_validate.pred_lm2), 2)


print("RMSE for Polynomial Model, degrees=2\nTraining/In-Sample: ", rmse_train, 
      "\nValidation/Out-of-Sample: ", rmse_validate)

print(f'Model Validate R2: {lm2_validate_r2}')



RMSE for Polynomial Model, degrees=2
Training/In-Sample:  208460.062284777 
Validation/Out-of-Sample:  210321.137540738
Model Validate R2: 0.2


Similar R2, but higher RMSE than best current model

---

In [198]:
# create the model object
glm = TweedieRegressor(power=1, alpha=0)


In [199]:

columns=['bedrooms', 'bathrooms', 'square_feet', 'age', 'county_Orange', 'county_Ventura']
# fit the model to our training data. 
glm.fit(X_train[columns], y_train.home_value)


TweedieRegressor(alpha=0, power=1)

In [204]:

# predict train
y_train['pred_glm'] = glm.predict(X_train[columns])

# evaluate: rmse
rmse_train = mean_squared_error(y_train.home_value, y_train.pred_glm)**(1/2)

glm_train_r2 = round(r2_score(y_train.home_value, y_train.pred_glm), 2)

# predict validate
y_validate['pred_glm'] = glm.predict(X_validate[columns])

# evaluate: rmse
rmse_validate = mean_squared_error(y_validate.home_value, y_validate.pred_glm)**(1/2)

# Evaluate R2

glm_validate_r2 = round(r2_score(y_validate.home_value, y_validate.pred_glm), 2)

print("RMSE for GLM using Tweedie, power=1 & alpha=0\nTraining/In-Sample: ", rmse_train, 
      "\nValidation/Out-of-Sample: ", rmse_validate)

print(f'Current Model train  R2: {glm_train_r2}')
print(f'Current Model Validate R2: {glm_validate_r2}')

RMSE for GLM using Tweedie, power=1 & alpha=0
Training/In-Sample:  232947.9993309968 
Validation/Out-of-Sample:  234612.5224126042
Current Model train  R2: 0.0
Current Model Validate R2: -0.0


In [ ]:
# Do a select * from the database and engineer features from there
import z_wrangle3

In [5]:
zillow = z_wrangle3.get_data_from_sql()

In [6]:
zillow.head()

,parcelid,id,logerror,transactiondate,id,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,11659079,76319,0.107525,2017-09-13,586346,NaN,NaN,NaN,3.0,5.0,...,NaN,NaN,112370.0,200780.0,2016.0,88410.0,2618.99,None,NaN,6.037262e+13
1,10914401,76320,0.023874,2017-09-13,1508459,NaN,NaN,NaN,2.0,2.0,...,NaN,NaN,141931.0,709658.0,2016.0,567727.0,8663.29,None,NaN,6.037144e+13
2,11211557,76324,-0.005895,2017-09-13,1076736,1.0,NaN,NaN,2.0,3.0,...,NaN,NaN,110241.0,137827.0,2016.0,27586.0,2602.30,None,NaN,6.037911e+13
3,11662195,76325,0.142955,2017-09-13,1719982,NaN,NaN,NaN,3.0,3.0,...,NaN,NaN,108779.0,1631733.0,2016.0,1522954.0,19587.90,None,NaN,6.037263e+13
4,12149213,76326,0.066862,2017-09-13,2720173,NaN,NaN,NaN,2.0,3.0,...,NaN,NaN,296521.0,862405.0,2016.0,565884.0,9604.06,None,NaN,6.037300e+13


Get NA counts to see what columns are usable

In [9]:
zillow.info(null_counts=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52275 entries, 0 to 52274
Data columns (total 62 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      52275 non-null  int64  
 1   id                            52275 non-null  int64  
 2   logerror                      52275 non-null  float64
 3   transactiondate               52275 non-null  object 
 4   id                            52275 non-null  int64  
 5   airconditioningtypeid         13628 non-null  float64
 6   architecturalstyletypeid      70 non-null     float64
 7   basementsqft                  47 non-null     float64
 8   bathroomcnt                   52275 non-null  float64
 9   bedroomcnt                    52275 non-null  float64
 10  buildingclasstypeid           0 non-null      object 
 11  buildingqualitytypeid         33709 non-null  float64
 12  calculatedbathnbr             52260 non-null  float64
 13  d

In [8]:
null1 = zillow.isnull().sum()
null1

parcelid                     0
id                           0
logerror                     0
transactiondate              0
id                           0
                         ...  
landtaxvaluedollarcnt        0
taxamount                    0
taxdelinquencyflag       50204
taxdelinquencyyear       50204
censustractandblock        114
Length: 62, dtype: int64

In [206]:
zillow.dtypes

parcelid                   int64
id                         int64
logerror                 float64
transactiondate           object
id                         int64
                          ...   
landtaxvaluedollarcnt    float64
taxamount                float64
taxdelinquencyflag        object
taxdelinquencyyear       float64
censustractandblock      float64
Length: 62, dtype: object